In [1]:
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import Point

block = pd.read_csv('data_census//tl_2010_06075_tabblock10.csv')
neigh_names = pd.read_csv('data_sf_open//Realtor_Neighborhoods.csv')

In [2]:
neigh_id = []

for index, row in block.iterrows():
    
    c = fiona.open("data_sf_open//geo_export_d30ee75b-b941-4798-962c-05f777bc2084.shx")
    pol = c.next()
    geom = shape(pol['geometry'])

    point_test = Point((row['INTPTLON10'],row['INTPTLAT10']))
    
    try:
        while geom.contains(point_test) == False:
            pol = c.next()
            geom = shape(pol['geometry'])
        neigh_id.append(pol['id'])
        
    except:
        neigh_id.append(np.nan)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  del sys.path[0]


In [26]:
neigh_dict = dict(zip([str(i) for i in neigh_names.index], neigh_names.nbrhood))
block_neigh = pd.DataFrame({'block_fips':block['GEOID10'], 'nbrhood_id':neigh_id})
block_neigh['nbrhood_name'] = block_neigh['nbrhood_id'].map(neigh_dict)

In [27]:
block_neigh.to_csv('block_neigh.csv', index = False)